# Criar agente que faz deep research

### Setando o modelo

In [76]:
from agents import Agent, Runner, trace,function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings

from openai import AsyncOpenAI

from crewai_tools import SerperDevTool

from pydantic import BaseModel, Field
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown

load_dotenv(find_dotenv())

True

In [53]:
client_groq = AsyncOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

model_llama4 = OpenAIChatCompletionsModel(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    openai_client=client_groq
)

model_llama = OpenAIChatCompletionsModel(
    model="llama3-8b-8192",
    openai_client=client_groq
)

model_deepseek = OpenAIChatCompletionsModel(
    model="deepseek-r1-distill-llama-70b",
    openai_client=client_groq
)

### Criando tool que busca na internet

In [74]:
@function_tool
def search_tool(query: str) -> str:
    """
    Usa a API da Serper para realizar uma busca no Google.
    
    Args:
        query: A consulta de busca.
        
    Returns:
        Uma string com os resultados da busca.
    """
    
    if not os.getenv("SERPER_API_KEY"):
        raise ValueError("A variável de ambiente 'SERPER_API_KEY' não está definida.")

    search = SerperDevTool(
        n_results=2
    )
    results = search._run(query=query)
    all_results_search = [f"Link: {result['link']} \nContent: {result['snippet']}\n\n" for result in results["organic"]]
    all_context = "\n\n".join(all_results_search)
    
    return all_context

### Criando um agente de busca

In [106]:
instructions_search = """
Você é um assistente de pesquisa. Dado um termo de busca, você busca na web por esse termo e
produz um resumo conciso dos resultados. Você receberá links e o conteúdo da página. Faça o output ser:

Links: [lista de links que você vai receber]
Resumo: Aqui vai o resumo sobre o assunto encotrado nos links acima.

O resumo deve ter cerca de 2-3 parágrafos e menos de 300 palavras. Você deve capturar os principais pontos.
O conteúdo resumido por ti, irá ser consumido por alguém que irá escrever alguma reportagem baseado no termo de busca. Então, é
de extrema importância você capturar a essência do resultado e ignorar pontos irrelevantes.
Não inclua algum comentário adicional além do resumo em si. Faça o resumo sempre em português do Brasil.
"""

In [107]:
search_agent = Agent(
    name="search agent",
    instructions=instructions_search,
    model=model_deepseek,
    tools=[search_tool],
    model_settings=ModelSettings(temperature=0)
)

### Rodando o agente

In [108]:
message = "frameworks de agentes mais recentes 2025"

with trace("search"):
    result = await Runner.run(starting_agent=search_agent, input=message)

In [109]:
display(Markdown(result.final_output))

Links: 
- https://medium.com/@elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239 
- https://www.shakudo.io/blog/top-9-ai-agent-frameworks 

Resumo:  
Os frameworks de agentes mais recentes em 2025 destacam-se por suas capacidades avançadas em processamento de linguagem natural e aprendizado de máquina. O GPT-5, da OpenAI, é apontado como um dos principais, com melhorias significativas em compreensão e geração de texto. Outros frameworks, como o MISTICO, do Meta, e o Llama 3, da Meta, também são mencionados por suas habilidades em tarefas complexas e multitarefa.  

Além disso, o Claude 3, da Anthropic, e o PaLM 3, do Google, são destacados por suas capacidades de generalização e integração com sistemas externos. Esses frameworks estão revolucionando a forma como os agentes de IA interagem com usuários e ambientes, permitindo soluções mais personalizadas e eficientes.

### Criando agente que faz output format

In [111]:
from typing import List
class OutputFormat(BaseModel):
    conteudo: str = Field(description="Conteúdo que foi baseado os links")
    links: List[str] = Field(description="Lista de links")

In [115]:
output_format_prompt = """
Você irá receber um conteúdo contendo uma lista de links e o resumo do conteúdo.
Sua tarefa é devolver o resultado no formato pydantic.
"""

output_format_agent = Agent(
    name="Output formarter agent",
    instructions=output_format_prompt,
    model=model_llama4,
    output_type=OutputFormat
)

In [116]:
final_result = await Runner.run(starting_agent=output_format_agent, input=result.final_output)

In [118]:
print(final_result.final_output)

conteudo='Os frameworks de agentes mais recentes em 2025 destacam-se por suas capacidades avançadas em processamento de linguagem natural e aprendizado de máquina. O GPT-5, da OpenAI, é apontado como um dos principais, com melhorias significativas em compreensão e geração de texto. Outros frameworks, como o MISTICO, do Meta, e o Llama3, da Meta, também são mencionados por suas habilidades em tarefas complexas e multitarefa. Além disso, o Claude3, da Anthropic, e o PaLM3, do Google, são destacados por suas capacidades de generalização e integração com sistemas externos. Esses frameworks estão revolucionando a forma como os agentes de IA interagem com usuários e ambientes, permitindo soluções mais personalizadas e eficientes.' links=['https://medium.com/@elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239', 'https://www.shakudo.io/blog/top-9-ai-agent-frameworks']
